In [18]:
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
from dash import dash_table
import pandas as pd
from pymongo import MongoClient
from bson.objectid import ObjectId

##############################
# Database Interaction Class #
##############################

class AnimalShelter:
    def __init__(self, username, password, db_name="AAC", host="nv-desktop-services.apporto.com", port=31116):
        self.client = MongoClient(f"mongodb://{username}:{password}@{host}:{port}/{db_name}?authSource={db_name}")
        self.db = self.client[db_name]
        self.collection = self.db["AnimalShelter"]

    def create_animal(self, animal_data):
        result = self.collection.insert_one(animal_data)
        return str(result.inserted_id)

    def read_animal(self, animal_id):
        animal = self.collection.find_one({"_id": ObjectId(animal_id)})
        return animal

    def update_animal(self, animal_id, updated_data):
        result = self.collection.update_one({"_id": ObjectId(animal_id)}, {"$set": updated_data})
        return result.modified_count

    def delete_animal(self, animal_id):
        result = self.collection.delete_one({"_id": ObjectId(animal_id)})
        return result.deleted_count

    def get_record_criteria(self, criteria):
        return list(self.collection.find(criteria))

    def list_animals(self):
        return list(self.collection.find())

################################
# Dash App and Data Management #
################################

# Initialize the database connection
username = "aacuser"
password = "green7"
shelter = AnimalShelter(username, password)

# Fetch data from MongoDB and convert it into a pandas DataFrame
def fetch_animal_data():
    try:
        animals = shelter.list_animals()
        # Convert MongoDB data to DataFrame and handle ObjectId conversion
        for animal in animals:
            animal['_id'] = str(animal['_id'])  # Convert ObjectId to string for display
        return pd.DataFrame(animals)
    except Exception as e:
        print(f"Error fetching animal data: {e}")
        return pd.DataFrame()  # Return an empty DataFrame on error

# Initialize Dash app
app = JupyterDash(__name__)

# Define the layout of the app
app.layout = html.Div(children=[
    html.H1(children='Animal Shelter Dashboard'),

    html.Div(children='''Welcome to the Animal Shelter Dashboard.'''),

    dcc.Dropdown(
        id='animal-dropdown',
        options=[
            {'label': 'All Animals', 'value': 'all'},
            # You can add more filters if necessary
        ],
        value='all',
    ),

    # Dash DataTable for displaying animal data
    dash_table.DataTable(
        id='animal-table',
        columns=[{'name': i, 'id': i} for i in fetch_animal_data().columns],
        data=fetch_animal_data().to_dict('records')
    ),
])

##################################
# Callback for updating the table #
##################################

@app.callback(
    Output('animal-table', 'data'),
    [Input('animal-dropdown', 'value')]
)
def update_table(filter_value):
    try:
        if filter_value == 'all':
            animals_df = fetch_animal_data()
        else:
            # You can add more filtering logic here if needed
            animals_df = fetch_animal_data()

        return animals_df.to_dict('records')
    except Exception as e:
        print(f"Error updating table: {e}")
        return []  # Return an empty list on error

###################
# Run the app     #
###################

if __name__ == '__main__':
    app.run_server(debug=True, mode='inline')  # Use 'inline' to run the app in the notebook

Error fetching animal data: Authentication failed., full error: {'ok': 0.0, 'errmsg': 'Authentication failed.', 'code': 18, 'codeName': 'AuthenticationFailed'}
Error fetching animal data: Authentication failed., full error: {'ok': 0.0, 'errmsg': 'Authentication failed.', 'code': 18, 'codeName': 'AuthenticationFailed'}


Error fetching animal data: Authentication failed., full error: {'ok': 0.0, 'errmsg': 'Authentication failed.', 'code': 18, 'codeName': 'AuthenticationFailed'}
